# **PMR3508 - Exercício Programa 02**: <br> **Redes Neurais e o Dataset MNIST**
---
---

## ✏️ **Cabeçalho**:

### **Nome**: `Vítor Garcia Comissoli`
### **NUSP**: `11810411`
### **Hash**: `109`

---

## 📜 **Descrição:**

Neste exercício, você irá trabalhar com o **dataset MNIST**, um conjunto de dados com 70.000 imagens de dígitos escritos à mão. Seu objetivo será aplicar os conceitos de **Redes Neurais Artificiais (ANNs)** vistos na aula teórica. Este EP está dividido em tarefas, sua formatação não deve ser alterada, mas novas células de código ou texto podem ser criadas nos blocos de cada tarefa.

---

## ⚠️ **Instruções:**
- Complete todas as tarefas abaixo, respondendo às perguntas e escrevendo o código necessário.
- Comente seu código para facilitar a correção.
- Entregue o notebook no formato `.ipynb`.

---


## ✅ **Tarefas:**

1. **Probabilidades de dígitos no *dataset***       ⇒ `2 pontos`
2. **Análise Exploratória de Dados (EDA)**          ⇒ `2 pontos`
3. **Treinamento e teste de Modelos**               ⇒ `2 pontos`
4. **Comunicação de Resultados e Visualizações**    ⇒ `2 pontos`
5. **Publicação no *Kaggle* e Documentação**        ⇒ `2 pontos`

---

<br>
<center>
        <h1>
        <b>
        BOA SORTE !!!
        </b>
        </h1>
</center>
<br>

---
---

## ⏳ Loading dos Dados

In [ ]:
import pickle
import random
import numpy as np
import os
from os.path import join

# Configuração de seeds para replicabilidade
np.random.seed(42)  # Seed para NumPy
random.seed(42)     # Seed para o módulo random

input_path = "../input/pmr3508-mnist"  # Obtém o diretório atual
images_filepath = join(input_path, 'MNIST-images.pkl')
labels_filepath = join(input_path, 'MNIST-labels.pkl')
validation_images_filepath = join(input_path, 'MNIST-validation-images.pkl')

with open(images_filepath, 'rb') as f:
    X_tot = pickle.load(f)

with open(labels_filepath, 'rb') as f:
    y_tot = pickle.load(f)

with open(validation_images_filepath, 'rb') as f:
    X_val = pickle.load(f)

In [ ]:
'''
TESTE DE CARREGAMENTO DO DATASET POR VISUALIZAÇÃO
Este bloco visualiza algumas imagens do dataset MNIST para verificar se o
carregamento foi realizado corretamente.
'''

%matplotlib inline
import random
import matplotlib.pyplot as plt

def show_images(images, title_texts):
    # Função para mostrar as imagens com seus respectivos títulos
    cols = 3  # Número de colunas na visualização
    rows = int(len(images) / cols) + 1  # Calcula o número de linhas
    plt.figure(figsize=(12, 12))  # Define o tamanho da figura
    index = 1
    for x in zip(images, title_texts):  # Itera sobre as imagens e títulos
        image = x[0]
        title_text = x[1]
        plt.subplot(rows, cols, index)  # Adiciona um subplot
        plt.axis('off')  # Desativa os eixos
        plt.imshow(image, cmap=plt.cm.gray)  # Mostra a imagem em escala de cinza
        if (title_text != ''):
            plt.title(title_text, fontsize=15)  # Define o título da imagem
        index += 1
    plt.tight_layout()  # Ajusta o layout para evitar sobreposição de títulos
    plt.show()  # Exibe a figura com as imagens e títulos

images_2_show = []  # Lista para armazenar as imagens a serem mostradas
titles_2_show = []  # Lista para armazenar os títulos das imagens
# Seleciona aleatoriamente 9 imagens de treino
for i in range(0, 9):
    r = random.randint(1, 60000)
    images_2_show.append(X_tot[r])  # Adiciona a imagem selecionada à lista
    titles_2_show.append(f"Imagem [{str(r)}] = {str(y_tot[r])}")  # Adiciona o título correspondente

show_images(images_2_show, titles_2_show)  # Exibe as imagens selecionadas

# 0️⃣ Suas bibliotecas & Constantes

In [ ]:
# Bibliotecas:

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Constantes:

random_state = 42
HASH = 109

# 1️⃣ Tarefa 01: Probabilidades 🎲

## 🧑🏻‍💻 Item a)

Descubra o número do Dataset associado ao seu Hash.

In [ ]:
X, y = shuffle(X_tot, y_tot, random_state = random_state)

In [ ]:
num = X[HASH]
label = y[HASH]

print("Número associado ao meu HASH: ", label, "\n")
# print("\n", "Matriz associada ao meu HASH: ", num)

In [ ]:
show_images([num], [f"Número associado ao meu HASH: {label}"])

## 🔦 Item b)

Determine, para a imagem vinculada ao seu Hash, qual é a Probabilidade de um píxel claro (128 - 255) para esta única imagem?

In [ ]:
min = 128; max = 255

claro = np.logical_and(num >= min, num <= max)

In [ ]:
probabilidade = 100 * (np.sum(claro)/num.size)

In [ ]:
print(f"A probabilidade de um pixel ser claro é de {probabilidade:.2f}%")

## ♟️ Item c)

Qual é a probabilidade de um píxel ser claro dentre todos os píxeis que tem a mesma classe que a sua imagem obtida em a)?

In [ ]:
imagem = X[y == label]

claro2 = np.logical_and(imagem >= min, imagem <= max)

In [ ]:
probabilidade2 = 100 * (np.sum(claro2)/imagem.size)

In [ ]:
print(f"A probabilidade de um pixel ser claro dada a imagem associada ao meu HASH é de {probabilidade2:.2f}%")

---

# 2️⃣ Tarefa 02: Análise Exploratória de Dados 📊

## ✨ Item a)

Conte quantas vezes cada dígito (de 0 a 9) aparece e responda:

1. Todos os dígitos aparecem a mesma quantidade?

2. Qual o valor médio dos píxeis de cada dígito?

In [ ]:
cont = np.bincount(y)

In [ ]:
plt.figure(figsize = (16, 5))
plt.bar(range(10), cont)
plt.title('Número de aparições de cada um dos Dígitos')
plt.xlabel('Dígitos')
plt.ylabel('Número de aparições')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, 9)

ax.set_xlim(left = -.5, right = 9.5)
_ = plt.xticks(range(10))

In [ ]:
print("Número de aparição de cada um dos Dígitos:", "\n")
for digit, freq in enumerate(cont):
    print(f"{digit}: Aparece em {freq} imagens")

### Resposta 01:

É possível observar que nem todos os Dígitos aparecem na mesma quantidade (por mais que alguns Dígitos aparentam apresentar uma frequência bem similar entre sí), onde vemos que o 5, por exemplo, aparece em 5376 imagens, enquanto o 1 aparece em 6725 imagens, o que resulta numa diferença de 1349 imagens, o que representa aproximadamente 25% da quantidade de imagens onde o Dígito 5 é encontrado, o que aparenta ser uma diferença considerável.

Entretabto, vale ressaltar que a variação em geral entre os dígitos não aparenta ser muito grande, e que a distribuição de probabilidade plotada acima é razoávelmente similar a uma distribuição uniforme.

### Resposta 02:

Calculou-se o valor médio dos pixels de cada Dígito abaixo:

In [ ]:
média = np.array([np.mean(X[y == digit]) for digit in range(10)])/X[0].size

In [ ]:
plt.figure(figsize = (16, 5))
plt.bar(range(10), média)
plt.title('Valor médio dos pixels por Dígito')
plt.xlabel('Dígitos')
plt.ylabel('Valor médio dos pixels')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, 9)

ax.set_xlim(left = -.5, right = 9.5)
_ = plt.xticks(range(10))

In [ ]:
print("Valor médio dos pixels em cada um dos Dígitos:", "\n")

for digit in range(10):
    média2 = np.mean(X[(y == digit)])
    print(f"{digit}: O valor médio dos pixels desse Dígito foi de {média2:.2f}")

## 📏 Item b)

Faça um histograma que mostre a distribuição dos valores dos píxeis para cada dígito. Há muitos valores que são “apagados” (ou seja, com valor 0) ou a distribuição dos valores é mais equilibrada entre os dígitos?

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
axes = axes.ravel()

for digit in range(10):
    digit_mask = y == digit
    digit_images = X[digit_mask]
    
    axes[digit].hist(digit_images.ravel(), bins = 50, density = True)
    axes[digit].set_title(f'Número {digit}')
    axes[digit].set_xlabel('Valor do pixel')
    axes[digit].set_ylabel('Densidade')
    axes[digit].spines['top'].set_visible(False)
    axes[digit].spines['right'].set_visible(False)
    axes[digit].spines['left'].set_position(('outward', 8))
    axes[digit].spines['bottom'].set_position(('outward', 8))
    axes[digit].spines['bottom'].set_bounds(0, 255)
    ax.set_xlim(left = -.5, right = 255.5)

plt.tight_layout()
plt.show()

A grande maioria dos pixels, em todos os Dígitos, podem ser considerados apagados (iguais a 0). O dígito 1 apresenta o maior número de valores apagados, e o Dígito 0 apresenta o menor (que ainda representa cerca de $\frac{3}{4}$ do total de pixels).

## 🤓 Item c)

Crie uma imagem para cada dígito (de 0 a 9) em que cada píxel dessa nova imagem representa a média do valor dos píxeis para aquela classe. Você consegue reconhecer os dígitos nas imagens criadas?

In [ ]:
imagens_médias = []
for digit in range(10):
    dígitos = X[y == digit]
    imagem_média = np.mean(dígitos, axis = 0)
    imagens_médias.append(imagem_média)

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(17, 6))
axes = axes.ravel()

for idx, imagem_média in enumerate(imagens_médias):
    axes[idx].imshow(imagem_média.reshape(28, 28), cmap = 'gray')
    axes[idx].axis('off')
    axes[idx].set_title(f'Número {idx}')

É possível observar que, mesmo após a obtenção da imagem média para cada Dígito (e aplicado o filtro em preto e branco), ainda é possível se distinguir os valores numéricos de cada um dos Dígitos a olho nú.

---

# 3️⃣ Tarefa 03: Treinamento e Teste de Modelos 🤖

## 📈 Item a)

Treine a ANN1 com 784 entradas, 8 neurônios na 1⁠ª camada oculta, 8 neurônios na 2⁠ª camada oculta e 10 saídas. Utilize 5 épocas para o treinamento. Use a biblioteca `scikit-learn`:

- Input Layer: 784 entradas (28x28);
- Hidden Layer 1: 8 neurônios;
- Hidden Layer 2: 8 neurônios;
- Output Layer: 10 saídas; (Classificador 0-9)
- Treine com 10 épocas.

In [ ]:
camadas_ocultas = (8, 8)
épocas = 10

In [ ]:
X_r = X.reshape(-1, 28 * 28)/255 # Reshaping X para que tenha 784 entradas

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_r, y, test_size = 0.3, random_state = random_state)

In [ ]:
modelo1 = MLPClassifier(hidden_layer_sizes = camadas_ocultas, max_iter = épocas, random_state = random_state, verbose = True, solver = "adam",
                        alpha = 1e-5, warm_start = True, early_stopping = False)

In [ ]:
modelo1.fit(X_train, y_train)

In [ ]:
print(f"Acurácia de treino: {100 * (modelo1.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo1.score(X_test, y_test)):.2f}%")

## 📉 Item b)

Treine a ANN2 com 784 entradas, 256 neurônios na 1⁠ª camada oculta, 256 neurônios na 2⁠ª camada oculta, 256 neurônios na 3ª camada oculta, 256 neurônios na 4ª camada oculta e 10 saídas. Utilize 20 épocas dessa vez. Use a biblioteca `scikit-learn`.

- Input Layer: 784 entradas (28x28);
- Hidden Layer 1: 256 neurônios;
- Hidden Layer 2: 256 neurônios;
- Hidden Layer 3: 256 neurônios;
- Hidden Layer 4: 256 neurônios;
- Output Layer: 10 saídas; (Classificação 0-9)
- Treine com 20 épocas.

In [ ]:
camadas_ocultas = (256, 256, 256, 256)
épocas = 20

In [ ]:
modelo2 = MLPClassifier(hidden_layer_sizes = camadas_ocultas, max_iter = épocas, random_state = random_state, verbose = True, solver = "adam",
                        alpha = 1e-5, warm_start = True, early_stopping = False)

In [ ]:
modelo2.fit(X_train, y_train)

In [ ]:
print(f"Acurácia de treino: {100 * (modelo2.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo2.score(X_test, y_test)):.2f}%")

## ⚙️ Item c)

Agora você treinará um novo modelo, mais adequado. Para isso, gere ao menos 5 configurações de redes neurais, variando o número de camadas ocultas, o número de neurônios e o número de épocas. As configurações devem estar intermediárias entre `[8, 8]` e `[256, 256, 256, 256]`.

Utilize a função `GridSearchCV` para realizar uma busca exaustiva pelos hiperparâmetros e encontre a configuração que oferece o melhor classificador, justificando sua escolha com base nas métricas de validação.

In [ ]:
lista_camadas_ocultas = [(256, 128, 64, 32), (128, 128, 128, 128), (128, 128, 128), (256, 256, 256), (256, 256)]
lista_épocas = [20, 50]

In [ ]:
parâmetros = {"solver": ["adam"], "alpha" : [1e-5], "hidden_layer_sizes" : lista_camadas_ocultas, "random_state" : [random_state],
              "max_iter" : lista_épocas, "verbose" : [True], "warm_start" : [True], 'early_stopping' : [False]}

In [ ]:
modelo_atual = MLPClassifier(random_state = random_state)

busca = GridSearchCV(modelo_atual, parâmetros, n_jobs = -1, cv = 3, scoring = "accuracy")

In [ ]:
busca.fit(X_train, y_train)

In [ ]:
melhores_parâmetros = busca.best_params_

print("Os melhores valores para os hiperparâmetros do modelo de redes neurais encontrados pela função GridSearchCV foram: \n \n", melhores_parâmetros)

In [ ]:
modelo3 = busca.best_estimator_

In [ ]:
print(f"Acurácia de treino: {100 * (modelo3.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo3.score(X_test, y_test)):.2f}%")

## 🔧 Item d)

Para os modelos treinados nas questões a) e b), além do classificador encontrado na questão c), compare o desempenho dos modelos, analisando se apresentam *underfitting* ou *overfitting*. Justifique com gráficos e análises.

Para comparar os 3 modelos gerados, serão plotadas e analisádas tanto a curva de aprendizagem do treino como as acurárias (de treino e teste) para cada um deles.

In [ ]:
plt.plot(modelo1.loss_curve_)
plt.title('Curva de aprendizagem do Modelo 1 ao longo do treino')
plt.xlabel('Iteração')
plt.ylabel('Perda')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, modelo1.n_iter_ - 1)

ax.set_xlim(left = 0, right = modelo1.n_iter_ - 1)
plt.show()

In [ ]:
print(f"Acurácia de treino: {100 * (modelo1.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo1.score(X_test, y_test)):.2f}%")

Observa-se que, para o Modelo 1, a curva de aprendizado ao longo do treino não aparentou chegar a valores estáveis de perda, o que indica que esse modelo foi cortado prematuramente, e teria melhores resultados com um maior número de épocas. Já quanto a suas acurácias, elas se mostraram as menores dentre os 3 modelos, entretanto foram acima de 90%, o que é um bom sinal.

In [ ]:
plt.plot(modelo2.loss_curve_)
plt.title('Curva de aprendizagem do Modelo 2 ao longo do treino')
plt.xlabel('Iteração')
plt.ylabel('Perda')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, modelo2.n_iter_ - 1)

ax.set_xlim(left = 0, right = modelo2.n_iter_ - 1)
plt.show()

In [ ]:
print(f"Acurácia de treino: {100 * (modelo2.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo2.score(X_test, y_test)):.2f}%")

Já para o Modelo 2, observa-se que a curva de aprendizado ao longo do treino chegou a valores um pouco mais estáveis de perda (mas ainda há variações perceptíveis entre os valores de cada iteração), o que implica que o número de iterações foi bem escolhido, mas um número um pouco maior provavelmente ainda chegaria em melhores resultados. Já quanto a suas acurácias, elas se mostraram melhores que as do Modelo 1, mas inferiores as do Modelo 3 (por bem pouco), já que a acurácia de teste foi de aproximadamente 97.81% o que é um ótimo resultado.

In [ ]:
plt.plot(modelo3.loss_curve_)
plt.title('Curva de aprendizagem do Modelo 3 ao longo do treino')
plt.xlabel('Iteração')
plt.ylabel('Perda')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, modelo3.n_iter_ - 1)

ax.set_xlim(left = 0, right = modelo3.n_iter_ - 1)
plt.show()

In [ ]:
print(f"Acurácia de treino: {100 * (modelo3.score(X_train, y_train)):.2f}%")
print(f"Acurácia de teste: {100 * (modelo3.score(X_test, y_test)):.2f}%")

Por fim, para o Modelo 3, observa-se que a curva de aprendizado ao longo do treino chegou a valores estáveis de perda (a partir de cerca de 33 iterações), o que implica que o número de iterações foi bem escolhido, e um número um pouco maior não chegaria em melhores resultados. Já quanto a suas acurácias, elas se mostraram melhores que as melhores dentre os 3 modelos (mas somente um pouco melhores que as do Modelo 2), já que a acurácia de teste foi de aproximadamente 97.93% o que é um ótimo resultado.

Vale ressaltar que a acurácia de testes ter sido igual a 100% pode ser um indicador da presença de overfitting, porém, como a acurácia de teste também se mostrou a mais alta entre os modelos, o modelo continua sendo o que melehor performa, e portanto, o mais adequado.

---

# 4️⃣ Tarefa 04: Resultados e Visualizações 🤞🏻

## 🐳 Item a)

Gere e apresente uma matriz de confusão que mostre a distribuição das previsões do melhor modelo. Quais as métricas de Acurácia, Precisão, Recall e F1-Score para esse modelo?

In [ ]:
y_pred = modelo3.predict(X_test)

In [ ]:
plt.figure(figsize=(18, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão')
plt.ylabel('Valor real')
plt.xlabel('Previsão')
plt.show()

In [ ]:
métricas = classification_report(y_test, y_pred, output_dict = True)
df_métricas = pd.DataFrame(métricas).transpose()

print("As métricas para o Modelo 3 foram: \n", df_métricas)

## 🐸 Item b)

Exiba gráficos que mostram a evolução da acurácia e da perda (`Loss`) durante o treinamento do melhor modelo encontrado no item 3c).

A evolução da perda surante o treinamento pode ser observada pelo mesmo gráfico usado no item 3d), plotado novamente abaixo:

In [ ]:
plt.plot(modelo3.loss_curve_)
plt.title('Curva de aprendizagem do Modelo 3 ao longo do treino')
plt.xlabel('Iteração')
plt.ylabel('Perda')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 8))
ax.spines['bottom'].set_bounds(0, modelo3.n_iter_ - 1)

ax.set_xlim(left = 0, right = modelo3.n_iter_ - 1)
plt.show()

Já para a evolução da acurácia, temos:

In [ ]:
épocas = 50
accuracy_history = []

for época in range(épocas):
    modelo3.fit(X_train, y_train)
    y_pred = modelo3.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracy_history.append(acc)

# Plotar a acurácia
plt.figure(figsize = (16, 6))
plt.plot(range(1, épocas + 1), accuracy_history, marker = 'o')
plt.title('Evolução da Acurácia Durante o Treinamento do Modelo 3')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.grid(True)
plt.legend()
plt.show()

## 🦖 Item c)

Escolha algumas imagens do conjunto de teste e mostre previsões do seu modelo, com acertos e erros. Discuta quais fatores podem ter contribuído para essas previsões corretas e incorretas.

In [ ]:
X_test_r = X_test.reshape(-1, 28, 28)

X_certo = X_test_r[y_test == y_pred]

In [ ]:
y_pred_certo = y_pred[y_test == y_pred]

y_certo = y_test[y_test == y_pred]

Seguem exemplos de previsões corretas realizadas pelo Modelo 3:

In [ ]:
show_images(X_certo[:12], [f'Predição = {y_pred_certo[k]}\n Valor real = {y_certo[k]}' for k in range(12)])

In [ ]:
X_errado = X_test_r[y_test != y_pred]

In [ ]:
y_pred_errado = y_pred[y_test != y_pred]

y_errado = y_test[y_test != y_pred]

Seguem exemplos de previsões incorretas realizadas pelo Modelo 3:

In [ ]:
show_images(X_errado[:12], [f'Predição = {y_pred_errado[k]}\n Valor real = {y_errado[k]}' for k in range(12)])

O principal fator que pode ter acarretado no erro de certas previsões se dá pela grafia não muito convencional (com uma grafia mais distorcida, menos clara e definida) dos dígitos que foram mal classificados, onde observam-se valores que até a olho nú não são rapidamente classificáveis.

Um exemplo se dá pelo dígito 2, que foi predito no modelo como 0, onde até um humano observando essa grafia poderia confundi-la com os números 0, 8 e 9.

Da mesma forma, grafias mais claras e definidas podem ser observadas nos exemplos retratados de acerto do modelo, onde os números são facilmente identificáveis a olho nú.

---

# 5️⃣ Tarefa 05: Lembrete *Kaggle* e Documentação 🗃️

## 😮‍💨 Item a)

Lembre-se de publicar no *Kaggle* com o título correto e padronizado!

## 🙏🏻 Item b)

Lembre-se de documentar adequadamente seu código e conclusões!

# Submissão das previsões

In [ ]:
with open(validation_images_filepath, 'rb') as f:
    X_test = pickle.load(f)

In [ ]:
X_train = X_tot.reshape(-1,28 * 28) / 255

y_train = y_tot

In [ ]:
camadas_ocultas = (256, 128, 64, 32)
épocas = 50

In [ ]:
modelo_final = MLPClassifier(hidden_layer_sizes = camadas_ocultas, max_iter = épocas, random_state = random_state, verbose = True, solver = "adam",
                        alpha = 1e-5, warm_start = True, early_stopping = False)

In [ ]:
modelo_final.fit(X_train, y_train)

In [ ]:
previsão = modelo_final.predict(X_test.reshape(-1,28 * 28) / 255)

#previsão
#len(previsão)

In [ ]:
df_submissão = pd.DataFrame({'ID': np.arange(1,10001), 'Answer':previsão})
df_submissão.set_index("ID", inplace = True)

In [ ]:
df_submissão.head()

In [ ]:
df_submissão.to_csv('submission.csv')